In [2]:
from pyspark.sql import SparkSession

# Replace with your Oracle connection details
jdbc_url = "jdbc:oracle:thin:@//localhost:1521/xe"
username = "HR"
password = "HR"

try:
    # Create a Spark session with JDBC options
    spark = SparkSession.builder.appName("JDBCConnectivityTest").getOrCreate()
    EMP_SRC_df = spark.read.jdbc(url=jdbc_url, table="EMP_SRC",  properties={"user": username, "password": password})
    EMP_PYSPARK_SCD2_df = spark.read.jdbc(url=jdbc_url, table="EMP_PYSPARK_SCD2",  properties={"user": username, "password": password})
    # If the connection was successful, you can print the result
    EMP_SRC_df.show()
    EMP_PYSPARK_SCD2_df.show()
   # spark.stop()
    print("JDBC connection test successful.")
except Exception as e:
    print(f"JDBC connection test failed: {str(e)}")

+-----+------+---------+----+-------------------+-------+-------+------+
|EMPNO| ENAME|      JOB| MGR|           HIREDATE|    SAL|   COMM|DEPTNO|
+-----+------+---------+----+-------------------+-------+-------+------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17 00:00:00|5000.00|   NULL|    10|
| 7698| BLAKE|  MANAGER|7839|1981-05-01 00:00:00|2850.00|   NULL|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09 00:00:00|2450.00|   NULL|    10|
| 7566| JONES|  MANAGER|7839|1981-04-02 00:00:00|2975.00|   NULL|    20|
| 7788| SCOTT|  ANALYST|7566|1987-04-19 00:00:00|3000.00|   NULL|    20|
| 7902|  FORD|  ANALYST|7566|1981-12-03 00:00:00|3000.00|   NULL|    20|
| 7369| SMITH|    CLERK|7902|1980-12-17 00:00:00| 800.00|   NULL|    20|
| 7499| ALLEN| SALESMAN|7698|1981-02-20 00:00:00|1600.00| 300.00|    30|
| 7521|  WARD| SALESMAN|7698|1981-02-22 00:00:00|1250.00| 500.00|    30|
| 7654|MARTIN| SALESMAN|7698|1981-09-28 00:00:00|1250.00|1400.00|    30|
| 7844|TURNER| SALESMAN|7698|1981-09-08 00:00:00|15

In [4]:
# Step 1 : Identify the records to update
from pyspark.sql.functions import col, lit, current_date
# Assuming EMP_SRC_df and EMP_PYSPARK_SCD2_df are already defined
# Step 1: Identify records to update
updates_df = EMP_SRC_df.alias("src") \
    .join(EMP_PYSPARK_SCD2_df.alias("tgt"), on="EMPNO",how="left_outer") \
    .filter(
        (col("src.JOB") != col("tgt.JOB")) |
        (col("src.SAL") != col("tgt.SAL")) |
        (col("src.COMM") != col("tgt.COMM")) |
        (col("src.MGR") != col("tgt.MGR")) |
        (col("src.DEPTNO") != col("tgt.DEPTNO"))
    ) \
    .select("src.EMPNO", "src.JOB", "src.SAL", "src.COMM", "src.MGR", "src.DEPTNO") \
    .withColumn("EFF_STRT_DT", current_date()) \
    .withColumn("EFF_END_DT", lit(None)) \
    .withColumn("MOST_RCNT_IN_CD", lit("Y")) \
    .withColumn("ETL_CHKSUM_NB", lit(None)) \
    .withColumn("ETL_LOAD_TS", current_date()) \
    .withColumn("ETL_LST_UPDT_TS", current_date())
    
updates_df.show()



+-----+---+---+----+---+------+-----------+----------+---------------+-------------+-----------+---------------+
|EMPNO|JOB|SAL|COMM|MGR|DEPTNO|EFF_STRT_DT|EFF_END_DT|MOST_RCNT_IN_CD|ETL_CHKSUM_NB|ETL_LOAD_TS|ETL_LST_UPDT_TS|
+-----+---+---+----+---+------+-----------+----------+---------------+-------------+-----------+---------------+
+-----+---+---+----+---+------+-----------+----------+---------------+-------------+-----------+---------------+

